# Growth Media

The availability of nutrients from the extracellular space has a major impact on the metabolic fluxes within the cell. COBRApy can be used to manage the exchanges between the external environment and your metabolic model. In experimental settings the "environment" is usually referred to as the growth medium, this implies all the concentrations of metabolites and co-factors available to the modeled organism. Unfortuantely, constraint-based metabolic models are based on fluxes, not concentrations. As a reminder, fluxes have the unit mmol/[gDW * h] (concentration (mmol) divided by grams dry weight of cells (gDW) times the time in hours (h)). 

Also, constraint-based modeling sets an upper bound for the particular import flux and not the flux itself. There are some crude approximations. As an examaple, if you supply 1 mol of glucose every 24h to 1 gram of bacteria you could set the upper exchange flux for glucose to 1 mol/[1 gDW * 24 h] since that is the nominal maximum that can be imported. There is no guarantee that all the glucose will be consumed with that flux. The preferred data for exchange fluxes are direct flux measurements obtained from timecourse exa-metabolome measurements. 

Setting the COBRApy environment

In [1]:
from pandas import DataFrame
import pandas as pd
pd.set_option('display.max_rows', 1000)

from cobrapy_bigg_client import client
model_orig = client.download_model('e_coli_core', save=False) # Download model from the BIGG database

Set parameter Username
Academic license - for non-commercial use only - expires 2022-10-10


## Nutrient Uptake

Let's start by looking at the exchanges that are open allowing flux to enter the cell.

In [2]:
model = model_orig.copy()
exchange_ids = [r.id for r in model.exchanges]
exchange_names = [r.name for r in model.exchanges]
exchange_formula = [r.reaction for r in model.exchanges]
exchange_lb = [r.lower_bound for r in model.exchanges]
exchange_ub = [r.upper_bound for r in model.exchanges]
exchangeList = {'Reaction ID': exchange_ids,
                'Reaction Name': exchange_names,
                'Reaction Formula': exchange_formula,
                'Reaction Lower Bound': exchange_lb,
                'Reaction Upper Bound': exchange_ub,
               }
exchange_df = pd.DataFrame(exchangeList, columns= ['Reaction ID','Reaction Name','Reaction Formula',
                                                   'Reaction Lower Bound','Reaction Upper Bound'])
open_exchanges = exchange_df[exchange_df['Reaction Lower Bound'] != 0] # removing reactions with lowerbound equal to zero
open_exchanges = open_exchanges.reset_index(drop=True) # renumbering the index
open_exchanges

Read LP format model from file C:\Users\hinton\AppData\Local\Temp\tmpfbrplvt_.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros


,Reaction ID,Reaction Name,Reaction Formula,Reaction Lower Bound,Reaction Upper Bound
0,EX_co2_e,CO2 exchange,co2_e <=>,-1000.0,1000.0
1,EX_glc__D_e,D-Glucose exchange,glc__D_e <=>,-10.0,1000.0
2,EX_h_e,H+ exchange,h_e <=>,-1000.0,1000.0
3,EX_h2o_e,H2O exchange,h2o_e <=>,-1000.0,1000.0
4,EX_nh4_e,Ammonia exchange,nh4_e <=>,-1000.0,1000.0
5,EX_o2_e,O2 exchange,o2_e <=>,-1000.0,1000.0
6,EX_pi_e,Phosphate exchange,pi_e <=>,-1000.0,1000.0


A simpler way of getting the minimum info is to use the "medium" method. Note that the lower bounds are positive instead of negative.

In [3]:
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

Put into a panda dataframe

In [4]:
open_exchanges = pd.DataFrame(list(model.medium.items()), columns = ['Exchange Reaction','Lower Bound'])
open_exchanges

,Exchange Reaction,Lower Bound
0,EX_co2_e,1000.0
1,EX_glc__D_e,10.0
2,EX_h_e,1000.0
3,EX_h2o_e,1000.0
4,EX_nh4_e,1000.0
5,EX_o2_e,1000.0
6,EX_pi_e,1000.0


If you want to block one of the open exchange reactions from allowing flux to enter the cell, you can do the following.

In [5]:
medium = model.medium
medium["EX_o2_e"] = 0.0
model.medium = medium

model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

As we can see oxygen import is now removed from the list of active exchanges and we can verify that this also leads to a lower growth rate.

In [6]:
model.slim_optimize()

0.21166294973531055

There is a small trap here. model.medium can not be assigned to directly. So the following will not work:

In [7]:
model.medium["EX_co2_e"] = 0.0
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

As you can see EX_co2_e is not set to zero. This is because model.medium is just a copy of the current exchange fluxes. Assigning to it directly with model.medium[..] = ... will not change the model. You have to assign an entire dictionary with the changed import flux upper bounds. It is always safer to use the traditional method shown below.

In [8]:
model.reactions.EX_co2_e.lower_bound = 0
model.medium

{'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

Setting the growth medium also connects to the context manager, so you can set a specific growth medium in a reversible manner.

In [9]:
model = model_orig.copy()

with model:
    medium = model.medium
    medium["EX_o2_e"] = 0.0
    model.medium = medium
    print(model.slim_optimize())
print(model.slim_optimize())
model.medium

Read LP format model from file C:\Users\hinton\AppData\Local\Temp\tmpoauj4et5.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros
0.21166294973531055
0.8739215069684302


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

So the medium change is only applied within the with block and reverted automatically.

## Secretion

Now let's look at the exchange reactions that will allow secretion into the extracellular space

In [10]:
model = model_orig.copy()
secreting_exchanges = exchange_df[exchange_df['Reaction Upper Bound'] != 0] # removing reactions with upper bound equal to zero
secreting_exchanges = secreting_exchanges.reset_index(drop=True) # renumbering the index
secreting_exchanges

Read LP format model from file C:\Users\hinton\AppData\Local\Temp\tmp9be9_lgb.lp
Reading time = 0.01 seconds
: 72 rows, 190 columns, 720 nonzeros


,Reaction ID,Reaction Name,Reaction Formula,Reaction Lower Bound,Reaction Upper Bound
0,EX_ac_e,Acetate exchange,ac_e -->,0.0,1000.0
1,EX_acald_e,Acetaldehyde exchange,acald_e -->,0.0,1000.0
2,EX_akg_e,2-Oxoglutarate exchange,akg_e -->,0.0,1000.0
3,EX_co2_e,CO2 exchange,co2_e <=>,-1000.0,1000.0
4,EX_etoh_e,Ethanol exchange,etoh_e -->,0.0,1000.0
5,EX_for_e,Formate exchange,for_e -->,0.0,1000.0
6,EX_fru_e,D-Fructose exchange,fru_e -->,0.0,1000.0
7,EX_fum_e,Fumarate exchange,fum_e -->,0.0,1000.0
8,EX_glc__D_e,D-Glucose exchange,glc__D_e <=>,-10.0,1000.0
9,EX_gln__L_e,L-Glutamine exchange,gln__L_e -->,0.0,1000.0


## Minimal Media

There are times when the smallest growth medium that can maintain a specific growth rate is needed, the so called "minimal medium". For this the function *minimal_medium* which by default obtains the medium with the lowest total import flux. This function needs two arguments: the model and the minimum growth rate (or other objective) the model has to achieve.

In [11]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth)

EX_glc__D_e    10.000000
EX_nh4_e        4.765319
EX_o2_e        21.799493
EX_pi_e         3.214895
dtype: float64

In this case the growth is limited by the glucose import.

Alternatively, a minimal medium with the smallest number of active imports is required. This can be achieved by using the *minimize_components* argument (this uses a MIP formulation and will therefore be much slower). The second argument of the method is the minimum value of the objective value that will be allowed. In the following case growth, the objective function, has to be greater that 0.1.

In [12]:
minimal_medium(model, 0.1, minimize_components=True)

EX_glc__D_e    6.874849
EX_nh4_e       0.545280
EX_pi_e        0.367870
dtype: float64

When minimizing the number of import fluxes there may be many alternative solutions. To obtain several of those you can also pass a positive integer to *minimize_components* which will give you at most that many alternative solutions. Let us try that with our model and also use the *open_exchanges* argument which will assign a large upper bound to all import reactions in the model. The return type will be a pandas.DataFrame.

In [13]:
minimal_medium(model, 0.8, minimize_components=8, open_exchanges=True)

,0,1,2,3,4,5
EX_fru_e,38.12664,0.00000,0.000000,496.362947,0.000000,0.00000
EX_glc__D_e,0.00000,496.83904,30.293984,0.000000,0.000000,0.00000
EX_gln__L_e,2.18112,2.18112,0.000000,0.000000,0.000000,0.00000
EX_glu__L_e,0.00000,0.00000,6.108400,0.000000,23.468185,496.52308
EX_mal__L_e,0.00000,0.00000,0.000000,0.000000,1000.000000,0.00000
EX_nh4_e,0.00000,0.00000,0.000000,4.362240,0.000000,0.00000
EX_o2_e,0.00000,0.00000,0.000000,0.000000,0.000000,500.00000
EX_pi_e,2.94296,2.94296,2.942960,2.942960,15.667461,2.94296


So there are 6 alternative solutions in total. One aerobic and five anaerobic solutions using different carbon sources.